In [45]:
input = r"""x00: 1
x01: 0
x02: 1
x03: 1
x04: 0
y00: 1
y01: 1
y02: 1
y03: 1
y04: 1

ntg XOR fgs -> mjb
y02 OR x01 -> tnw
kwq OR kpj -> z05
x00 OR x03 -> fst
tgd XOR rvg -> z01
vdt OR tnw -> bfw
bfw AND frj -> z10
ffh OR nrd -> bqk
y00 AND y03 -> djm
y03 OR y00 -> psh
bqk OR frj -> z08
tnw OR fst -> frj
gnj AND tgd -> z11
bfw XOR mjb -> z00
x03 OR x00 -> vdt
gnj AND wpb -> z02
x04 AND y00 -> kjc
djm OR pbm -> qhw
nrd AND vdt -> hwm
kjc AND fst -> rvg
y04 OR y02 -> fgs
y01 AND x02 -> pbm
ntg OR kjc -> kwq
psh XOR fgs -> tgd
qhw XOR tgd -> z09
pbm OR djm -> kpj
x03 XOR y03 -> ffh
x00 XOR y04 -> ntg
bfw OR bqk -> z06
nrd XOR fgs -> wpb
frj XOR qhw -> z04
bqk OR frj -> z07
y03 OR x01 -> nrd
hwm AND bqk -> z03
tgd XOR rvg -> z12
tnw OR pbm -> gnj
"""

In [46]:
# part 1

import re
from enum import StrEnum

class Operator(StrEnum):
    AND = 'AND'
    OR = 'OR'
    XOR = 'XOR'

from typing import NamedTuple
Rule = NamedTuple('Rule', [('operator', Operator), ('input1', str), ('input2', str), ('output', str)])

regex = r"(don't\(\)|do\(\)|mul\(\d+,\d+\))"

assignRegex = r"(\w+): (\d)"
ruleRegex = r"(\w+) (AND|OR|XOR) (\w+) -> (\w+)"

registers: dict[str, int] = {}
rules: list[Rule] = []
for line in input.splitlines():
    if (line == ""):
        continue
    result = re.search(assignRegex, line) 
    if (result != None):
        registers[result.group(1)] = int(result.group(2))
        continue
    result = re.search(ruleRegex, line) 
    if (result != None):
        rules.append(Rule(input1 = result.group(1), operator = result.group(2), input2 = result.group(3), output = result.group(4)))
        continue
    print (f"Not matched: {line}")
    raise NotImplementedError

print (len(registers), len(rules))

while True:
    numAssigned = 0
    for rule in rules:
        if (rule.output in registers): continue
        if (rule.input1 not in registers): continue
        if (rule.input2 not in registers): continue

        numAssigned += 1
        if (rule.operator == Operator.AND):
            registers[rule.output] = registers[rule.input1] & registers[rule.input2]
        elif (rule.operator == Operator.OR):
            registers[rule.output] = registers[rule.input1] | registers[rule.input2]
        elif (rule.operator == Operator.XOR):
            registers[rule.output] = registers[rule.input1] ^ registers[rule.input2]
        else:
            print (rule.operator)
            raise NotImplementedError
    if (numAssigned == 0):
        break

z = 0
for registername, binaryvalue in registers.items():
    if (registername[0] != 'z'): continue
    registernum = int(registername[1:])
    z = z | (binaryvalue << registernum)

print (z)






10 36
2024


In [47]:
# part 2

# Old, inefficient 6-gate approach
# x01 XOR y01 -> a01
# a01 XOR carry01 -> z01
# x01 OR y01 -> b01
# b01 AND carry01 -> d01
# x01 AND y01 -> e01
# d01 OR e01 -> carry02


# 46 z outputs, 45 x/y inputs

# 222 rules -> 2 for initial half-adder, + 44 * 5 per adder


# 0, 0, 0 -> a01 = 0, z01 = 0, carry02 = 0
# 0, 0, 1 -> a01 = 0, z01 = 1, carry02 = 0
# 1, 0, 0 -> a01 = 1, z01 = 1, carry02 = 0
# 1, 0, 1 -> a01 = 1, z01 = 0, carry02 = 1
# 1, 1, 1 -> a01 = 0, z01 = 1, carry02 = 1

# 2-bit adder
# x00 AND y00 -> carry01
# x00 XOR y00 -> z00

# Full adder: two XOR, two AND, one OR

# x01 XOR y01 -> a01
# a01 XOR carry01 -> z01
# a01 AND carry01 -> b01
# x01 AND y01 -> d01
# b01 OR d01 -> carry02

## z02 = carry02

import re
from enum import StrEnum

class Operator(StrEnum):
    AND = 'AND'
    OR = 'OR'
    XOR = 'XOR'

from typing import NamedTuple
Rule = NamedTuple('Rule', [('operator', Operator), ('input1', str), ('input2', str), ('output', str)])

regex = r"(don't\(\)|do\(\)|mul\(\d+,\d+\))"

assignRegex = r"(\w+): (\d)"
ruleRegex = r"(\w+) (AND|OR|XOR) (\w+) -> (\w+)"

swaps = [('frn', 'z05'),
         ('wnf', 'vtj'),
         ('gmq', 'z21'),
         ('wtt', 'z39')]

sortedswaps = []
for swap in swaps:
    sortedswaps.append(swap[0])
    sortedswaps.append(swap[1])
sortedswaps = sorted(sortedswaps)
print (*sortedswaps, sep=",")

def applySwaps(str):
    for swap in swaps:
        if (str == swap[0]): return swap[1]
        if (str == swap[1]): return swap[0]
    return str

registers: dict[str, int] = {}
rules: list[Rule] = []
for line in input.splitlines():
    if (line == ""):
        continue
    result = re.search(assignRegex, line) 
    if (result != None):
        registers[result.group(1)] = int(result.group(2))
        continue
    result = re.search(ruleRegex, line) 
    if (result != None):
        rules.append(Rule(
            input1 = result.group(1),
            operator = result.group(2),
            input2 = result.group(3),
            output = applySwaps(result.group(4))))
        continue
    print (f"Not matched: {line}")
    raise NotImplementedError


adderRules: list[Rule] = []
ruleSequence = []
regNameMap = {}

def inputsAre(rule: Rule, input1, input2):
    mappedInput1 = input1
    mappedInput2 = input2
    if (input1[0] != 'x' and input1[0] != 'y'):
        mappedInput1 = regNameMap[input1]
    if (input2[0] != 'x' and input2[0] != 'y'):
        mappedInput2 = regNameMap[input2]
    if (rule.input1 == mappedInput1 and rule.input2 == mappedInput2): return True
    if (rule.input1 == mappedInput2 and rule.input2 == mappedInput1): return True
    return False

def oneInputIs(rule: Rule, input):
    mappedInput = input
    if (input[0] != 'x' and input[0] != 'y'):
        mappedInput = regNameMap[input]
    if (rule.input1 == mappedInput or rule.input2 == mappedInput): return True
    return False

# find first half-adder
for rule in rules:
    if (inputsAre(rule, 'x00', 'y00')):
        if (rule.operator == 'AND'):
            regNameMap['carry01'] = rule.output
            ruleSequence.append((rule, 'carry01'))
            rules.remove(rule)
        elif (rule.operator == 'XOR'):
            if (rule.output != "z00"): raise NotImplementedError
            regNameMap['z00'] = rule.output
            ruleSequence.append((rule, 'z00'))
            rules.remove(rule)
        else: raise NotImplementedError

print (len(rules))

def findNextInstruction(rule, currentBit, operator, prefix1, prefix2, outputprefix):
    bitStr = str(currentBit).zfill(2)
    found = False
    input1 = f'{prefix1}{bitStr}'
    input2 = f'{prefix2}{bitStr}'
    outputName = f'{outputprefix}{bitStr}'
    if (outputprefix == 'carry'):
        outputName = f'carry{str(currentBit+1).zfill(2)}'
    for rule in rules:
        if (inputsAre(rule, input1, input2) and rule.operator == operator):
            regNameMap[outputName] = rule.output
            ruleSequence.append((rule, outputName))
            rules.remove(rule)
            found = True
            break
    if (not found):
        print (regNameMap)
        print (f"Not found at currentBit: {currentBit}, '({input1} {operator} {input2} -> {outputName})")
        for rule in rules:
            for input in [input1, input2]:
                if (oneInputIs(rule, input) and rule.operator == operator):
                    print (f"Candidate rule containing {input} ({regNameMap.get(input)}): {rule}")
        raise NotImplementedError


targetBit = 44
currentBit = 0
instruction = 0
while currentBit != targetBit:
    # 0: x01 XOR y01 -> a01
    # 1: x01 AND y01 -> d01
    # 2: a01 XOR carry01 -> z01
    # 3: a01 AND carry01 -> b01
    # 4: b01 OR d01 -> carry02

    ## z02 = carry02

    currentBit += 1
    if (instruction == 0):
        findNextInstruction(rule, currentBit, 'XOR', 'x', 'y', 'a')
        instruction += 1
    if (instruction == 1):
        findNextInstruction(rule, currentBit, 'AND', 'x', 'y', 'd')
        instruction += 1
    if (instruction == 2):
        findNextInstruction(rule, currentBit, 'XOR', 'a', 'carry', 'z')
        instruction += 1
    if (instruction == 3):
        findNextInstruction(rule, currentBit, 'AND', 'a', 'carry', 'b')
        instruction += 1
    if (instruction == 4):
        findNextInstruction(rule, currentBit, 'OR', 'b', 'd', 'carry')
        instruction = 0

frn,gmq,vtj,wnf,wtt,z05,z21,z39
36
{}
Not found at currentBit: 1, '(x01 XOR y01 -> a01)


NotImplementedError: 